In [37]:
from data import Data
from dimension_reduction import PCADimensionReduction
from utils import display_autoencoder_metrics

from keras.models import Sequential
from keras.layers import Dense

In [38]:
data_filepath = "data/SC_integration/counts_ctc_simulated_123_5k.tsv"
true_results_filepath = "data/SC_integration/ids_ctc_simulated_123_5k.tsv"
healthy_train_indices_filepath = "data/SC_integration/healthy_train_indices.npy"
healthy_test_indices_filepath = "data/SC_integration/healthy_test_indices.npy"

SEED = 42
FOLD_NUMBER = 3

# You can change these values to work better for models
CUT_BY_MAX_THRESHOLD = 4
PCA_VARIABLES_AMOUNT = 60

In [39]:
data_object = Data(data_filepath, true_results_filepath)

healthy_train_data, healthy_test_data, cancer_data =  data_object.get_train_test_healthy_split_cancer_data(healthy_train_indices_filepath, healthy_test_indices_filepath)
scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_data = data_object.get_scaled_healthy_train_test_cancer_data()

In [40]:
cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_data = data_object.get_cut_by_max_healthy_train_test_cancer_data(CUT_BY_MAX_THRESHOLD)

pca_object = PCADimensionReduction(cut_by_max_healthy_train_data, scaled_healthy_train_data, data_object.train_true_results, SEED)
pca_variables = pca_object.get_most_important_variables_from_pc1(PCA_VARIABLES_AMOUNT)
pca_reduced_healthy_train_data = cut_by_max_healthy_train_data[pca_variables.index]
pca_reduced_healthy_test_data = cut_by_max_healthy_test_data[pca_variables.index]
pca_reduced_cancer_data = cut_by_max_cancer_data[pca_variables.index]

In [41]:
def create_autoencoder(input_shape):
    model = Sequential()
    model.add(Dense(1000, input_dim=input_shape, activation='relu'))
    model.add(Dense(500, activation='relu')) # size to compress to
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(input_shape))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

## Autoencoder

### Regular data

In [42]:
autoencoder = create_autoencoder(healthy_train_data.shape[1])
autoencoder.fit(healthy_train_data, healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1322
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1143
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1089
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.1036
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0986
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0952
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0923
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0897
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0872
Epoch 10/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0856
Epoch 11/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0842
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0829
Epoch 13/100


In [43]:
display_autoencoder_metrics(autoencoder, healthy_train_data, healthy_test_data, cancer_data)

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.2469018769592623
Test RMSE: 0.34020545621509285
Cancer data RMSE: 0.5450958578847498


### Scaled data

In [44]:
autoencoder = create_autoencoder(scaled_healthy_train_data.shape[1])
autoencoder.fit(scaled_healthy_train_data, scaled_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 1ms/step - loss: 0.9211
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8994
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8896
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8782
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8763
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8563
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8403
Epoch 8/100
190/190 [==============================] - 0s 2ms/step - loss: 0.8310
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.8129
Epoch 10/100
190/190 [==============================] - 0s 2ms/step - loss: 0.8001
Epoch 11/100
190/190 [==============================] - 0s 1ms/step - loss: 0.7840
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 0.7666
Epoch 13/100


In [45]:
display_autoencoder_metrics(autoencoder, scaled_healthy_train_data, scaled_healthy_test_data, scaled_cancer_data)

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.7125836506939516
Test RMSE: 0.9494251561971796
Cancer data RMSE: 6.559844003766266


### Cut by max data

In [46]:
autoencoder = create_autoencoder(cut_by_max_healthy_train_data.shape[1])
autoencoder.fit(cut_by_max_healthy_train_data, cut_by_max_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 985us/step - loss: 0.4490
Epoch 2/100
190/190 [==============================] - 0s 991us/step - loss: 0.0852
Epoch 3/100
190/190 [==============================] - 0s 991us/step - loss: 0.0319
Epoch 4/100
190/190 [==============================] - 0s 995us/step - loss: 0.0191
Epoch 5/100
190/190 [==============================] - 0s 984us/step - loss: 0.0156
Epoch 6/100
190/190 [==============================] - 0s 994us/step - loss: 0.0125
Epoch 7/100
190/190 [==============================] - 0s 999us/step - loss: 0.0095
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0080
Epoch 9/100
190/190 [==============================] - 0s 997us/step - loss: 0.0079
Epoch 10/100
190/190 [==============================] - 0s 995us/step - loss: 0.0077
Epoch 11/100
190/190 [==============================] - 0s 999us/step - loss: 0.0058
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 

In [47]:
display_autoencoder_metrics(autoencoder, cut_by_max_healthy_train_data, cut_by_max_healthy_test_data, cut_by_max_cancer_data)

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.05394235689897675
Test RMSE: 0.0695097391855808
Cancer data RMSE: 0.17538660760323424


### PCA reduced

In [48]:
autoencoder = create_autoencoder(pca_reduced_healthy_train_data.shape[1])
autoencoder.fit(pca_reduced_healthy_train_data, pca_reduced_healthy_train_data, batch_size=16, epochs=100)

Epoch 1/100
190/190 [==============================] - 0s 1ms/step - loss: 0.4460
Epoch 2/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0633
Epoch 3/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0265
Epoch 4/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0143
Epoch 5/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0140
Epoch 6/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0075
Epoch 7/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0084
Epoch 8/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0080
Epoch 9/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0066
Epoch 10/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0108
Epoch 11/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0077
Epoch 12/100
190/190 [==============================] - 0s 1ms/step - loss: 0.0057
Epoch 13/100


In [49]:
display_autoencoder_metrics(autoencoder, pca_reduced_healthy_train_data, pca_reduced_healthy_test_data, pca_reduced_cancer_data)

2/2 [==============================] - 0s 1ms/step
Train RMSE: 0.08836637308646005
Test RMSE: 0.10162575061382327
Cancer data RMSE: 0.1989458956637712
